In [1]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path("..")
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

In [2]:
skills_demand_path = DATA_PROCESSED / "skills_demand.parquet"
skills_demand = pd.read_parquet(skills_demand_path)

skills_demand.shape, skills_demand.columns

((105320, 3), Index(['jobId', 'year', 'skill'], dtype='object'))

In [3]:
onet_skills_path = DATA_RAW / "onet" / "db_30_1_text" / "Skills.txt"

onet_skills = pd.read_csv(
    onet_skills_path,
    sep="\t",
    dtype="string"
)

onet_skills.shape, onet_skills.columns

((62580, 13),
 Index(['O*NET-SOC Code', 'Element ID', 'Element Name', 'Scale ID',
        'Data Value', 'N', 'Standard Error', 'Lower CI Bound', 'Upper CI Bound',
        'Recommend Suppress', 'Not Relevant', 'Date', 'Domain Source'],
       dtype='object'))

In [4]:
im = onet_skills[onet_skills["Scale ID"] == "IM"].copy()

onet_skill_catalog = (
    im[["Element ID", "Element Name"]]
    .drop_duplicates()
    .sort_values("Element Name")
    .reset_index(drop=True)
)

onet_skill_catalog.shape, onet_skill_catalog.head(10)

((35, 2),
   Element ID                  Element Name
 0    2.A.2.b               Active Learning
 1    2.A.1.b              Active Listening
 2    2.B.2.i       Complex Problem Solving
 3    2.B.1.b                  Coordination
 4    2.A.2.a             Critical Thinking
 5    2.B.3.j         Equipment Maintenance
 6    2.B.3.c           Equipment Selection
 7    2.B.3.d                  Installation
 8    2.B.1.e                   Instructing
 9    2.B.4.e  Judgment and Decision Making)

In [5]:
onet_skill_catalog["skill_norm"] = (
    onet_skill_catalog["Element Name"]
    .str.strip()
    .str.lower()
)

onet_skill_catalog.head(10)

,Element ID,Element Name,skill_norm
0,2.A.2.b,Active Learning,active learning
1,2.A.1.b,Active Listening,active listening
2,2.B.2.i,Complex Problem Solving,complex problem solving
3,2.B.1.b,Coordination,coordination
4,2.A.2.a,Critical Thinking,critical thinking
5,2.B.3.j,Equipment Maintenance,equipment maintenance
6,2.B.3.c,Equipment Selection,equipment selection
7,2.B.3.d,Installation,installation
8,2.B.1.e,Instructing,instructing
9,2.B.4.e,Judgment and Decision Making,judgment and decision making


In [6]:
skills_in_onet = set(skills_demand["skill"])
onet_skills_set = set(onet_skill_catalog["skill_norm"])

len(skills_in_onet & onet_skills_set), sorted(skills_in_onet & onet_skills_set)

(11,
 ['coordination',
  'equipment maintenance',
  'installation',
  'mathematics',
  'monitoring',
  'negotiation',
  'programming',
  'science',
  'time management',
  'troubleshooting',
  'writing'])

In [7]:
skills_demand_onet_flagged = skills_demand.copy()

skills_demand_onet_flagged["is_onet_skill"] = (
    skills_demand_onet_flagged["skill"]
    .isin(onet_skill_catalog["skill_norm"])
)

skills_demand_onet_flagged["is_onet_skill"].value_counts()

is_onet_skill
False    103552
True       1768
Name: count, dtype: int64

In [8]:
skills_onet_subset = skills_demand_onet_flagged[
    skills_demand_onet_flagged["is_onet_skill"]
].copy()

skills_onet_subset.shape

(1768, 4)

In [9]:
onet_by_year = (
    skills_onet_subset
    .groupby("year")
    .size()
    .sort_index()
)

onet_by_year

year
2017      11
2018       5
2019       6
2020      16
2021      16
2022      68
2023     202
2024    1444
dtype: int64

In [10]:
total_by_year = (
    skills_demand
    .groupby("year")
    .size()
    .sort_index()
)

total_by_year

year
2015       65
2016      159
2017      525
2018      412
2019      851
2020      799
2021     1192
2022     3440
2023     9058
2024    88819
dtype: int64

In [11]:
onet_coverage = (
    total_by_year
    .rename("total_skills")
    .to_frame()
    .join(onet_by_year.rename("onet_skills"), how="left")
    .fillna(0)
    .astype({"onet_skills": "int64"})
)

onet_coverage

,total_skills,onet_skills
year,,
2015,65,0
2016,159,0
2017,525,11
2018,412,5
2019,851,6
2020,799,16
2021,1192,16
2022,3440,68
2023,9058,202


In [12]:
onet_coverage["onet_share"] = (
    onet_coverage["onet_skills"] / onet_coverage["total_skills"]
)

onet_coverage

,total_skills,onet_skills,onet_share
year,,,
2015,65,0,0.000000
2016,159,0,0.000000
2017,525,11,0.020952
2018,412,5,0.012136
2019,851,6,0.007051
2020,799,16,0.020025
2021,1192,16,0.013423
2022,3440,68,0.019767
2023,9058,202,0.022301


In [13]:
out_path = DATA_PROCESSED / "onet_coverage_by_year.parquet"
onet_coverage.to_parquet(out_path)

out_path

WindowsPath('../data/processed/onet_coverage_by_year.parquet')